# GEMM (Using SYCL C++ Buffers)

The following example shows a simple matrix multiplication program using __SYCL C++__ with the __buffer/accessor__ style of memory management. Follow along with the instructions of the lab to build and run the program. The lab requires a mixture of observing key components, and making simple modifications.

In [1]:
%%writefile lab/dpcpp_gemm_buffers.cpp
//==============================================================
// Copyright © 2023 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <iostream>
#include <vector>
#include <sycl/sycl.hpp>          //# sycl namespace
#include "oneapi/mkl/blas.hpp"  //# oneMKL DPC++ interface for BLAS functions

//# The following project performs matrix multiplication using oneMKL / DPC++ with buffers.
//# We will execute the simple operation A * B = C
//# The matrix B is set equal to the identity matrix such that A * B = A * I
//# After performing the computation, we will verify A * I = C -> A = C

using namespace sycl;
namespace mkl = oneapi::mkl;  //# shorten mkl namespace

int main() {

    //# dimensions
    
    int m = 3, n = 3, k = 3;
    
    //# leading dimensions
    
    int ldA = 3, ldB = 3, ldC = 3;
    
    //# scalar multipliers
    
    float alpha = 1.0, beta = 1.0;
    
    //# transpose status of matrices
    
    mkl::transpose transA = mkl::transpose::nontrans;
    mkl::transpose transB = mkl::transpose::nontrans;
    
    //# matrix data
    
    std::vector<float> A = {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0};
    std::vector<float> B = {1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0};
    std::vector<float> C = {0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0};

    //### Step 1 - Create a queue with default selector.
    
    queue q;
    device my_device = q.get_device();
    std::cout << "Device: " << my_device.get_info<info::device::name>() << "\n";

    //### Step 2 - Create buffers to hold our matrix data.
    //# Buffer objects can be constructed given a container
    //# Observe the creation of buffers for matrices A and B.
    //# Try and create a third buffer for matrix C called C_buffer.
    //# The solution is shown in the hidden cell below.
    
    buffer A_buffer(A);
    buffer B_buffer(B);
    /* define C_buffer here */
    
    //### Step 3 - Execute gemm operation.
    //# Here, we need only pass in our queue and other familiar matrix multiplication parameters.
    //# This includes the dimensions and data buffers for matrices A, B, and C.
    
    mkl::blas::gemm(q, transA, transB, m, n, k, alpha, A_buffer, ldA, B_buffer, ldB, beta, C_buffer, ldC);


    //### Step 6 - Observe creation of accessors to retrieve data from A_buffer and C_buffer.
    
    host_accessor A_acc(A_buffer, read_only);
    host_accessor C_acc(C_buffer, read_only);

    int status = 0;

    // verify C matrix using accessor to observe values held in C_buffer
    
    std::cout << std::endl;
    std::cout << "C = " << std::endl;
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            if (A_acc[i*m+j] != C_acc[i*m+j]) status = 1;
            std::cout << C_acc[i*m+j] << " ";
        }
        std::cout << std::endl;
    }
    std::cout << std::endl;

    status == 0 ? std::cout << "Verified: A = C" << std::endl : std::cout << "Failed: A != C" << std::endl;
    return status;
}

Writing lab/dpcpp_gemm_buffers.cpp


### Solutions - click the three dots below to reveal

Step 4 - The correct line is
```sycl::buffer C_buffer(C);```

### Build and Run
Select the cell below and click Run ▶ to compile and execute the code above:

In [ ]:
! chmod 755 q; chmod 755 run_gemm_buffers.sh;if [ -x "$(command -v qsub)" ]; then ./q run_gemm_buffers.sh; else ./run_gemm_buffers.sh; fi

The build instructions for this sample can be found in the ```run_gemm_buffers.sh``` script. Consider using the [Link Line Advisor](https://software.intel.com/content/www/us/en/develop/articles/intel-mkl-link-line-advisor.html) to help you create compile and link lines for your oneMKL projects.

<html><body><span style="color:green"><h1>Survey</h1></span></body></html>

[We would appreciate any feedback you’d care to give, so that we can improve the overall training quality and experience. Thanks! ](https://intel.az1.qualtrics.com/jfe/form/SV_cCpY08ARDi6NhfT)

<html><body><span style="color:Red"><h1>Reset Notebook</h1></span></body></html>

##### Should you be experiencing any issues with your notebook or just want to start fresh run the below cell.

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
button = widgets.Button(
    description='Reset Notebook',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will update this notebook, overwriting any changes.',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          !rsync -a --size-only /data/oneapi_workshop/Intel_oneAPI_MKL_Training/00_GEMM/ ~/Intel_oneAPI_MKL_Training/00_GEMM/
          print('Notebook reset -- now click reload on browser.')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])